In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import duckdb as db 
import plotly.express as px
import country_converter as coco



In [3]:
customers = pd.read_csv('/Users/denistkachenko/Documents/accounts.csv', on_bad_lines='skip')
transactions = pd.read_csv('/Users/denistkachenko/Documents/transactions.csv', on_bad_lines='skip')


FileNotFoundError: [Errno 2] No such file or directory: '/Users/denistkachenko/Documents/accounts.csv'

In [80]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499927 entries, 0 to 499926
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   D8669246619153820  499927 non-null  object 
 1   IP123TL            499882 non-null  object 
 2   Unnamed: 2         413964 non-null  object 
 3   GB                 499906 non-null  object 
 4   253                499927 non-null  int64  
 5   FALSE              499927 non-null  bool   
 6   0                  499920 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.4+ MB


In [81]:
customers.columns = ['account_id', 'code', 'state', 'country', 'registered_at', 'active', 'balance']
customers.head()


,account_id,code,state,country,registered_at,active,balance
0,C1995222694378703,V3B 8G5,BC,CA,210,False,102.820139
1,E8265358916432007,K7P 3C4,ON,CA,1978,False,0.000000
2,G6871719269988452,19401,PA,US,2000,False,30.381250
3,X8144956966945241,45613,OH,US,1,False,0.000000
4,N1788167187349074,18901,PA,US,1676,False,1670.053472


In [82]:
transactions.head()

,I4127227587819694,8UX5SHHQ-FVEE-5XU7-FBF8-82OINYYK7KSD,1873.26,USD,6,A,P,173.225,connecticut,6840,...,6840.1,CT,US,6840.2,CT.1,US.1,M,0,17,3/8/2007 18:03
0,E9525730529761877,7LF834E4-BLWQ-FJ92-YEYA-85AF05RI2PCX,740.66,USD,13,A,P,81.159,greater london,sw2,...,GU11 1YH,NaN,GB,GU11 1YH,NaN,GB,M,10,19,7/26/2007 2:36
1,Z6376643485803420,8ZFFW0QS-PN3Y-9N5U-28DW-6HFI162RGVVF,1535.15,USD,4,A,P,97.118,colorado,80126,...,80104,CO,US,80104,CO,US,M,6,24,4/7/2007 19:39
2,R8609290025411138,SVV7LJ8A-PG2L-JVNJ-66NF-HDSQ107UQO5N,1324.41,USD,13,A,P,216.150,new york,14534,...,85233,AZ,US,85233,AZ,US,M,9,7,5/16/2007 11:37
3,Y9572700698484288,3SWVJA0R-MZ11-RM1T-E0BJ-XACU1NOQQRNC,697.31,BRL,14,A,P,91.119,wien,1000,...,1220,NaN,AT,1220,NaN,AT,M,0,25,10/18/2007 12:52
4,W9847720367646182,IG442ZIB-TJCR-V83L-0L8J-RHLM8MWX8MFK,575.16,USD,10,A,P,75.115,florida,33578,...,NaN,NaN,US,NaN,NaN,US,M,8,4,2/11/2007 1:59


In [83]:
transactions = transactions.drop(transactions.columns[[4,5,6,7,8,9,10,15,16,17,18,19,20,21,22,23]], axis=1)
transactions.columns = ['account_id', 'transaction_id', 'amount','currency', 'state', 'locale' , 'method', 'type', 'created_at']
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599999 entries, 0 to 599998
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   account_id      599999 non-null  object 
 1   transaction_id  599999 non-null  object 
 2   amount          599999 non-null  float64
 3   currency        599999 non-null  object 
 4   state           592688 non-null  object 
 5   locale          599999 non-null  object 
 6   method          599999 non-null  object 
 7   type            543182 non-null  object 
 8   created_at      599999 non-null  object 
dtypes: float64(1), object(8)
memory usage: 41.2+ MB


In [84]:
transactions['created_at'] = pd.to_datetime(transactions['created_at'])

In [85]:
customers['registered_at'] = pd.to_datetime(customers['registered_at'])

In [86]:
data = pd.merge(customers, transactions, left_on='account_id', right_on='account_id')

In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 599917 entries, 0 to 599916
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   account_id      599917 non-null  object        
 1   code            599861 non-null  object        
 2   state_x         496655 non-null  object        
 3   country         599884 non-null  object        
 4   registered_at   599917 non-null  datetime64[ns]
 5   active          599917 non-null  bool          
 6   balance         599909 non-null  float64       
 7   transaction_id  599917 non-null  object        
 8   amount          599917 non-null  float64       
 9   currency        599917 non-null  object        
 10  state_y         592606 non-null  object        
 11  locale          599917 non-null  object        
 12  method          599917 non-null  object        
 13  type            543105 non-null  object        
 14  created_at      599917 non-null  dat

In [88]:
data['week transaction'] = data['created_at'].dt.to_period('W').apply(lambda r: r.start_time)
data['month transaction'] = data['created_at'].dt.to_period('M').apply(lambda r: r.start_time)
data['day'] = data['created_at'].apply(lambda r: r.day_name())

In [89]:
data['day'].value_counts()

Saturday     134181
Sunday       110817
Wednesday     71768
Tuesday       71270
Monday        71130
Thursday      70859
Friday        69892
Name: day, dtype: int64

In [90]:
data['country'].value_counts()

US    406410
GB     35385
CA     28347
AU     19539
FR     13245
       ...  
AO         1
AG         1
WS         1
ET         1
ML         1
Name: country, Length: 174, dtype: int64

In [117]:
result1 = data.groupby(['country', 'week transaction'])['amount'].sum().reset_index()


In [120]:
result2 = result1.duplicated()
result2

0       False
1       False
2       False
3       False
4       False
        ...  
4332     True
4333     True
4334     True
4335     True
4336     True
Length: 4337, dtype: bool

In [92]:
pivot_table = result1.pivot(index='country', columns='week transaction', values='amount')


In [93]:
pivot_table = pivot_table.fillna(0)
pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174 entries, AD to ZW
Data columns (total 51 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   2007-02-05 00:00:00  174 non-null    float64
 1   2007-02-12 00:00:00  174 non-null    float64
 2   2007-02-19 00:00:00  174 non-null    float64
 3   2007-02-26 00:00:00  174 non-null    float64
 4   2007-03-05 00:00:00  174 non-null    float64
 5   2007-03-12 00:00:00  174 non-null    float64
 6   2007-03-19 00:00:00  174 non-null    float64
 7   2007-03-26 00:00:00  174 non-null    float64
 8   2007-04-02 00:00:00  174 non-null    float64
 9   2007-04-09 00:00:00  174 non-null    float64
 10  2007-04-16 00:00:00  174 non-null    float64
 11  2007-04-23 00:00:00  174 non-null    float64
 12  2007-04-30 00:00:00  174 non-null    float64
 13  2007-05-07 00:00:00  174 non-null    float64
 14  2007-05-14 00:00:00  174 non-null    float64
 15  2007-05-21 00:00:00  174 non-null    float64


In [94]:
data2 = customers.groupby('country')['account_id'].count()
data2.drop('US', inplace=True)

data2.index = coco.convert(names=data2.index, to='name_short', not_found=None)

In [95]:
data2.drop(['Canada', 'United Kingdom', 'Australia', 'France'], inplace=True)

In [96]:
fig = px.choropleth(data2, locations=data2.index, color=data2.values,
                    locationmode="country names",
                    color_continuous_scale=px.colors.sequential.Plasma)
fig['layout'].update(title='Number of accounts by country', width=2000, height=1600)
fig.show()

In [97]:
customers['country'].fillna('Other', inplace=True)

In [98]:
test2= customers.loc[customers['country'] == 'Other']

test2

,account_id,code,state,country,registered_at,active,balance
4391,O2824490450302628,98104,WA,Other,1970-01-01 00:00:00.000000001,False,0.000000
6598,Y6593627664008931,9000,Khomas,Other,1970-01-01 00:00:00.000000001,False,0.000000
61157,Q2199501138547222,-,NaN,Other,1970-01-01 00:00:00.000000001,False,0.000000
104736,E2028597697913336,-,NaN,Other,1970-01-01 00:00:00.000000001,False,0.000000
128596,R1486570527744516,98104,WA,Other,1970-01-01 00:00:00.000000001,False,0.000000
159657,H5373885776432378,9000,NaN,Other,1970-01-01 00:00:00.000000001,False,0.000000
162639,J9230275629348817,-,NaN,Other,1970-01-01 00:00:00.000000001,False,0.000694
193649,L1771886748488023,-,NaN,Other,1970-01-01 00:00:00.000000001,False,0.000000
205238,Q5216524624355000,9000,Namibia,Other,1970-01-01 00:00:00.000000001,False,0.000694
208407,I8704147776138976,-,NaN,Other,1970-01-01 00:00:00.000000001,False,0.000000


In [99]:
data.head()

,account_id,code,state_x,country,registered_at,active,balance,transaction_id,amount,currency,state_y,locale,method,type,created_at,week transaction,month transaction,day
0,C1995222694378703,V3B 8G5,BC,CA,1970-01-01 00:00:00.000000210,False,102.820139,B5ZIV4W1-DV54-3MJ4-NE3D-D3XF8WPPOD8A,996.65,USD,False,en-US,CREDITCARD,VISA,2007-03-02 17:42:00,2007-02-26,2007-03-01,Friday
1,C1995222694378703,V3B 8G5,BC,CA,1970-01-01 00:00:00.000000210,False,102.820139,8KNQXDAR-F9MI-04NU-0AHW-Y48PGYX3HKVT,853.26,USD,False,en-GB,CREDITCARD,VISA,2007-03-26 13:26:00,2007-03-26,2007-03-01,Monday
2,E8265358916432007,K7P 3C4,ON,CA,1970-01-01 00:00:00.000001978,False,0.000000,M81IWMAR-I4J7-Z29S-BXB7-8AAU3RZSKXL9,1597.25,USD,False,en-US,CREDITCARD,VISA,2007-05-17 11:00:00,2007-05-14,2007-05-01,Thursday
3,E8265358916432007,K7P 3C4,ON,CA,1970-01-01 00:00:00.000001978,False,0.000000,L9KKPR9T-ZS0V-A3ER-TD90-970KCD41F83M,496.97,USD,False,en-US,CREDITCARD,VISA,2007-04-18 05:33:00,2007-04-16,2007-04-01,Wednesday
4,E8265358916432007,K7P 3C4,ON,CA,1970-01-01 00:00:00.000001978,False,0.000000,8IGNM18I-NMZC-2O67-J9TM-6G668HXRH4RL,1075.40,USD,False,en-US,CREDITCARD,VISA,2007-04-23 14:36:00,2007-04-23,2007-04-01,Monday


In [100]:
#data.to_csv('/Users/denistkachenko/Documents/data.csv', index=False)